In [1]:
desc_folder = './results'
colId = 'Compound Name'

desc_rdkit = True
desc_fps = True
desc_cx = True
desc_custom = True

descType_list = []
if desc_custom:
    descType_list.append('custom')
if desc_rdkit:
    descType_list.append('rdkit')
if desc_fps:
    descType_list.append('fingerprints')
if desc_cx:
    descType_list.append('chemaxon')
descType_list
print(f"\tSelected descriptors types: {descType_list}")

	Selected descriptors types: ['custom', 'rdkit', 'fingerprints', 'chemaxon']


In [2]:
## load all descriptor tables and merge together
import pandas as pd
dataTable_merged = pd.DataFrame(columns=[colId])

for descType in descType_list:
    dataTable = pd.read_csv(f'{desc_folder}/descriptors_{descType}.csv')
    print(f"\tThere are total <{dataTable.shape[1]-1}> {descType} descriptors for <{dataTable.shape[0]}> molecules")
    dataTable_merged = dataTable_merged.merge(right=dataTable, on='Compound Name', how='outer')

print(f"\tThe merged data table has <{dataTable_merged.shape[0]}> molecules and <{dataTable_merged.shape[1]-1}> descriptors")

##
Descriptors = dataTable_merged.iloc[:, 1:]
print(Descriptors.shape)
dataTable_merged.head(5)

	There are total <2> custom descriptors for <38> molecules
	There are total <140> rdkit descriptors for <38> molecules
	There are total <2048> fingerprints descriptors for <38> molecules
	There are total <33> chemaxon descriptors for <38> molecules
	The merged data table has <38> molecules and <2223> descriptors


,Compound Name,custDesc_ADME MDCK(WT) Permeability;Mean;A to B Recovery (%),custDesc_ADME MDCK(WT) Permeability;Mean;B to A Papp (10^-6 cm/s);(Num),rd_MaxAbsEStateIndex,rd_MaxEStateIndex,rd_MinAbsEStateIndex,rd_MinEStateIndex,rd_qed,rd_SPS,rd_MolWt,...,cx_topology-analyser_chiralCenterCount,cx_topology-analyser_fsp3,cx_topology-analyser_fusedAliphaticRingCount,cx_topology-analyser_fusedAromaticRingCount,cx_topology-analyser_fusedRingCount,cx_topology-analyser_heteroRingCount,cx_topology-analyser_heteroAromaticRingCount,cx_topology-analyser_largestRingSize,cx_topology-analyser_largestRingSystemSize,cx_topology-analyser_rotatableBondCount
0,KT-0001038,85.597209,3.665047,14.199143,14.199143,0.014939,-3.035663,0.031658,18.025641,1080.072,...,3,0.403846,0,0,2,6,3,6,2,26
1,KT-0001174,83.548787,10.866813,13.198821,13.198821,0.029994,-4.445789,0.063074,15.200000,840.773,...,1,0.378378,0,0,2,5,3,6,2,21
2,KT-0001190,58.710000,8.920000,14.034131,14.034131,0.070255,-2.933381,0.057877,15.442623,836.901,...,1,0.372093,0,2,2,5,4,6,2,19
3,KT-0001674,65.396574,1.327664,14.863094,14.863094,0.132763,-0.719197,0.238176,20.085106,650.708,...,2,0.484848,0,2,2,4,1,6,2,11
4,KT-0001675,66.234937,8.333819,14.856497,14.856497,0.215315,-0.684509,0.229648,20.434783,636.725,...,2,0.515152,0,2,2,4,1,6,2,12


In [3]:
Descriptors = dataTable_merged.iloc[:, 1:]
print(Descriptors.shape)

(38, 2223)


In [4]:
def create_variance_table(list_desc, variance_filter):    
    variance_list = variance_filter.variances_.tolist()
    variance_cutoff = variance_filter.get_params(deep=True)['threshold']

    variance_dict = {}
    if len(list_desc) != len(variance_list):
        print(f"Warning! The desc list (N={len(list_desc)}) doesn't match the variance data (N={len(variance_list)})")
    else:
        for i in range(len(list_desc)):
            variance_dict[i] = {}
            variance_dict[i]['Descriptor'] = list_desc[i]
            variance_dict[i]['Variance'] = variance_list[i]
            variance_dict[i]['Select'] = 'Yes' if variance_list[i] > variance_cutoff else 'No'
    
    variance_table = pd.DataFrame.from_dict(variance_dict).T

    count_Sele = variance_table['Select'].value_counts().get('Yes', 0)
    count_Drop = variance_table['Select'].value_counts().get('No', 0)
    print(f"\tIn total <{len(list_desc)}> desc, there are <{count_Sele}> selected, <{count_Drop}> dropped, cutoff is {variance_cutoff}.")
    return variance_table

## Removing features with no variance
from sklearn.feature_selection import VarianceThreshold
variance_threshold = 0
variance_filter = VarianceThreshold(threshold=variance_threshold)
variance_filter.fit_transform(Descriptors)

##
variance_table = create_variance_table(list_desc=list(Descriptors.columns), variance_filter=variance_filter)
variance_table.head(3)

	In total <2223> desc, there are <938> selected, <1285> dropped, cutoff is 0.


,Descriptor,Variance,Select
0,custDesc_ADME MDCK(WT) Permeability;Mean;A to ...,80.682451,Yes
1,custDesc_ADME MDCK(WT) Permeability;Mean;B to ...,13.612443,Yes
2,rd_MaxAbsEStateIndex,0.298926,Yes
